In [1]:
import urllib.request
import urllib.parse
import json
import pprint
from pprint import PrettyPrinter

# === Step 1: API トークンの取得 ===
token_url = 'http://localhost:18080/kabusapi/token'
auth_data = { 'APIPassword': '1995taka' }  # 自身の API パスワードを設定
json_auth_data = json.dumps(auth_data).encode('utf8')

token_req = urllib.request.Request(token_url, data=json_auth_data, method='POST')
token_req.add_header('Content-Type', 'application/json')

try:
    with urllib.request.urlopen(token_req) as token_res:
        print("トークン取得レスポンス:", token_res.status, token_res.reason)
        token_content = json.loads(token_res.read())
        pprint.pprint(token_content)
except urllib.error.HTTPError as e:
    print("トークン取得 HTTP エラー:", e)
    try:
        error_content = json.loads(e.read())
        pprint.pprint(error_content)
    except Exception as parse_err:
        print("エラー解析中に問題発生:", parse_err)
    exit(1)
except Exception as e:
    print("トークン取得 エラー:", e)
    exit(1)

# API トークンの抽出
# ここに到達する前に、上記で exit(1) が呼ばれるため、token_content が定義されていない状態は避けられるはずです。
api_token = token_content.get('Token')
if not api_token:
    print("トークンが取得できませんでした。")
    exit(1)

# === Step 2: 注文履歴の取得 ===
orders_url = 'http://localhost:18080/kabusapi/orders'
params = { 'product': 0 }  # 必要に応じてパラメータを設定
query_string = urllib.parse.urlencode(params)
full_orders_url = f"{orders_url}?{query_string}"

orders_req = urllib.request.Request(full_orders_url, method='GET')
orders_req.add_header('Content-Type', 'application/json')
orders_req.add_header('X-API-KEY', api_token)  # 取得したトークンをヘッダーに設定

try:
    with urllib.request.urlopen(orders_req) as orders_res:
        print("\n注文履歴取得レスポンス:", orders_res.status, orders_res.reason)
        for header in orders_res.getheaders():
            print(header)
        print()
        orders_content = json.loads(orders_res.read())
        # pprint.pprint(orders_content)
        # 最新4件を抽出
        latest4_orders = orders_content[-4:]  # 古い順の場合。新しい順なら [:4] を使用
        print("\n最新4件の注文:")
        # pprint.pprint(latest4_orders)
        pp = PrettyPrinter(sort_dicts=False)
        pp.pprint(latest4_orders)
except urllib.error.HTTPError as e:
    print("注文履歴取得 HTTP エラー:", e)
    try:
        error_content = json.loads(e.read())
        pprint.pprint(error_content)
    except Exception as parse_err:
        print("エラー応答解析エラー:", parse_err)
except Exception as e:
    print("その他のエラー:", e)


トークン取得レスポンス: 200 OK
{'ResultCode': 0, 'Token': 'c38ad629055341e09188188d2d5b0224'}

注文履歴取得レスポンス: 200 OK
('Content-Length', '239984')
('Content-Type', 'application/json; charset=utf-8')
('Server', 'Microsoft-HTTPAPI/2.0')
('X-Content-Type-Options', 'nosniff')
('X-XSS-Protection', '1; mode=block')
('X-Frame-Options', 'DENY')
('Content-Security-Policy', "default-src 'none'")
('Access-Control-Allow-Origin', '*')
('Date', 'Wed, 22 Jan 2025 01:15:10 GMT')
('Connection', 'close')


最新4件の注文:
[{'CashMargin': 2,
  'MarginTradeType': 3,
  'MarginPremium': 0.0,
  'ID': '20250122A01N02326098',
  'State': 5,
  'OrderState': 5,
  'OrdType': 1,
  'RecvTime': '2025-01-22T09:40:20.9580916+09:00',
  'Symbol': '9432',
  'SymbolName': '日本電信電話',
  'Exchange': 1,
  'ExchangeName': '東証プ',
  'Price': 0.0,
  'OrderQty': 100.0,
  'CumQty': 100.0,
  'Side': '1',
  'AccountType': 4,
  'DelivType': 0,
  'ExpireDay': 20250122,
  'Details': [{'SeqNum': 1,
               'ID': '20250122A01N02326098',
               'R